##LSTM

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# monta G-drive en entorno COLAB
from google.colab import drive
drive.mount('/content/drive/')

file_path3= '/content/drive/MyDrive/productos_a_predecir.txt'
productos_predecir = pd.read_csv(file_path3, sep='\t')

file_path = '/content/drive/MyDrive/sell-in.txt'
sell_in = pd.read_csv(file_path, sep='\t')

In [2]:
file_path3= "C:/Users/vgarciario/desktop/UA MASTER/labo3/productos_a_predecir.txt"
productos_predecir = pd.read_csv(file_path3, sep='\t')

file_path = 'C:/Users/vgarciario/desktop/UA MASTER/labo3/sell-in.txt'
sell_in = pd.read_csv(file_path, sep='\t')

In [3]:
# Definir los periodos de interés
#periodos = [201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912]
periodos = [201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912]
# Filtrar las filas correspondientes a los periodos de interés
sell_in_completo = sell_in[sell_in['periodo'].isin(periodos)]

In [4]:
sell_in_completo.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [5]:
dataset = pd.merge(productos_predecir, sell_in_completo, on='product_id', how='inner')

In [6]:
sell_in_completo = dataset.groupby(['periodo', 'product_id'])['tn'].sum().reset_index()

In [7]:
sell_in_completo['periodo'] = pd.to_datetime(sell_in_completo['periodo'].astype(str) + '01', format='%Y%m%d')

In [8]:
sell_in_completo.set_index("periodo", inplace=True)

In [ ]:
sell_in_completo.to_csv('/content/drive/MyDrive/sell_in_completo.csv', index=False)

In [44]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
import keras_tuner as kt

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función del modelo para Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                 activation='relu'),
                            input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mse')
    return model

# Inicializar el tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_loss',
                        max_trials=5,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

# Variable para guardar los mejores hiperparámetros por producto
mejores_hiperparametros = []

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Dividir datos en conjuntos de entrenamiento y validación
        #split = int(0.8 * len(X))
        #X_train, X_val = X[:split], X[split:]
        #y_train, y_val = y[:split], y[split:]

        # Buscar los mejores hiperparámetros
        #tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=0)
        #best_hp = tuner.get_best_hyperparameters()[0]
        #mejores_hiperparametros.append({'product_id': producto, 'units': best_hp.get('units'), 'learning_rate': best_hp.get('learning_rate')})

        # Construir el mejor modelo
        #model = tuner.hypermodel.build(best_hp)

        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), verbose=0)

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)
mejores_hiperparametros_df = pd.DataFrame(mejores_hiperparametros)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)
mejores_hiperparametros_df.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/mejores_hiperparametros.csv", index=False)


ModuleNotFoundError: No module named 'sklearn'

In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
import keras_tuner as kt

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función del modelo para Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                 activation='relu'),
                            input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mse')
    return model

# Inicializar el tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_loss',
                        max_trials=5,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

# Variable para guardar los mejores hiperparámetros por producto
mejores_hiperparametros = []

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Dividir datos en conjuntos de entrenamiento y validación
        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        # Buscar los mejores hiperparámetros
        tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=0)
        best_hp = tuner.get_best_hyperparameters()[0]
        mejores_hiperparametros.append({'product_id': producto, 'units': best_hp.get('units'), 'learning_rate': best_hp.get('learning_rate')})

        # Construir el mejor modelo
        model = tuner.hypermodel.build(best_hp)

        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), verbose=0)

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)
mejores_hiperparametros_df = pd.DataFrame(mejores_hiperparametros)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)
mejores_hiperparametros_df.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/mejores_hiperparametros.csv", index=False)


ModuleNotFoundError: No module named 'tensorflow'

In [48]:
pip install -U scikit-learn

     ---------------------------------------- 11.0/11.0 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 3.3 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install -U tensorflow


     ------------------------------------ 377.0/377.0 MB 721.9 kB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
     ------------------------------------ 127.7/127.7 kB 197.7 kB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
     ---------------------------------------- 4.1/4.1 MB 4.0 MB/s eta 0:00:00
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
     ---------------------------------------- 1.5/1.5 MB 1.5 MB/s eta 0:00:00
  Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Using cached Markdown-3.6-py3-none-any.whl (105 kB)
  Using cached tensorboard_data_se

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
pip install keras

^C
Note: you may need to restart the kernel to use updated packages.


In [56]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf
import keras_tuner as kt

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", MinMaxScaler.__module__)
print("TensorFlow version:", tf.__version__)
print("Keras Tuner version:", kt.__version__)



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Numpy version: 1.26.4
Pandas version: 2.2.2
Scikit-learn version: sklearn.preprocessing._data
TensorFlow version: 2.16.1
Keras Tuner version: 1.4.7


In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
import keras_tuner as kt

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función del modelo para Keras Tuner
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                                 activation='relu'),
                            input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mse')
    return model

# Inicializar el tuner
tuner = kt.RandomSearch(build_model,
                        objective='val_loss',
                        max_trials=5,
                        executions_per_trial=3,
                        directory='my_dir',
                        project_name='intro_to_kt')

# Variable para guardar los mejores hiperparámetros por producto
mejores_hiperparametros = []

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Dividir datos en conjuntos de entrenamiento y validación
        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        # Buscar los mejores hiperparámetros
        tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), verbose=0)
        best_hp = tuner.get_best_hyperparameters()[0]
        mejores_hiperparametros.append({'product_id': producto, 'units': best_hp.get('units'), 'learning_rate': best_hp.get('learning_rate')})

        # Construir el mejor modelo
        model = tuner.hypermodel.build(best_hp)

        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), verbose=0)

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)
mejores_hiperparametros_df = pd.DataFrame(mejores_hiperparametros)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)
mejores_hiperparametros_df.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/mejores_hiperparametros.csv", index=False)


Reloading Tuner from my_dir\intro_to_kt\tuner0.json


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


...

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#VER
 #walk-forward validation for univariate data
#def walk_forward_validation(data, n_test, cfg):
#predictions = list()
# split dataset
#train, test = train_test_split(data, n_test)
# fit model
#model = model_fit(train, cfg)
# seed history with training dataset
#history = [x for x in train]
# step over each time step in the test set
##for i in range(len(test)):
# fit model and make forecast for history
#yhat = model_predict(model, history, cfg)
# store forecast in list of predictions
#predictions.append(yhat)
# add actual observation to history for the next loop
#history.append(test[i])
# estimate prediction error
#error = measure_rmse(test, predictions)
#print(' > %.3f' % error)
#return error

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=128, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Dividir datos en conjuntos de entrenamiento y validación
        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), verbose=0)

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(n_steps, n_features)))  # Reducido a 64 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, callbacks=[early_stopping], verbose=0)  # Reducido a 50 épocas

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Asegúrate de que productos_unicos está definido
productos_unicos = sell_in_completo["product_id"].unique()

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(n_steps, n_features)))  # Reducido a 64 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

for producto in productos_unicos:
    # Filtrar datos para el producto específico
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values

    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, callbacks=[early_stopping], verbose=0)  # Reducido a 50 épocas

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyboardInterrupt: 

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(n_steps, n_features)))  # Reducido a 64 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Filtrar datos para los productos top 30 y el resto
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

# Función para entrenar y predecir para un producto específico
def entrenar_y_predecir(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, callbacks=[early_stopping], verbose=0)  # Reducido a 50 épocas

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir(aux, producto)

# Procesar el resto de productos en conjunto
aux_restante = sell_in_completo[sell_in_completo["product_id"].isin(productos_restantes)].drop(columns=["product_id"]).values
if len(aux_restante) > 0:
    entrenar_y_predecir(aux_restante, 'resto_productos')

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step
         product_id  prediccion_1
0             20001   1375.632446
1             20002   1364.875244
2             20003    572.358765
3             20005    373.496490
4             20004    432.450195
5             20009    380.624298
6             20032    242.714554
7             20006    264.982544
8             20007    311.919739
9             20010    201.683884
10            20008    203.133987
11            20013    194.817078
12            20014    266.597382
13            20011    283.869598
14            20015    275.500519
15            20012    173.304413
16            20019    218.027817
17            20016    179.966843
18            20017    167.291290
19            20020    210.309723
20            20022    145.839905
21            20018    130.666382
22            20023    178.232727
23            20021    187.849243
24            20026    114.289543
25            20027    155.248383
26            20025    142.782654
27       

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(n_steps, n_features)))  # Reducido a 64 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico
def entrenar_y_predecir(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, callbacks=[early_stopping], verbose=0)  # Reducido a 50 épocas

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir(aux, producto)

# Procesar el resto de productos en conjunto pero predecir individualmente
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

for producto in productos_restantes:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir(aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo
def build_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=64, activation='relu'), input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico
def entrenar_y_predecir(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, callbacks=[early_stopping], verbose=0)

        # Preparar los datos para la predicción
        # Tomar la penúltima secuencia de datos para predecir dos períodos adelante
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir(aux, producto)

# Entrenar un único modelo para el resto de productos
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]
aux_restante = productos_restantes.drop(columns=["product_id"]).values
scaler.fit(aux_restante)
aux_restante_normalizado = scaler.transform(aux_restante)

X_restante, y_restante = crear_secuencias(aux_restante_normalizado, n_steps, step_ahead=2)
X_restante = X_restante.reshape((X_restante.shape[0], X_restante.shape[1], n_features))

# Construir y entrenar el modelo
modelo_restante = build_model()
modelo_restante.fit(X_restante, y_restante, epochs=50, callbacks=[early_stopping], verbose=0)

# Predecir para cada producto restante
for producto in productos_restantes["product_id"].unique():
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    aux_normalizado = scaler.transform(aux)
    
    # Preparar los datos para la predicción
    ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))
    prediccion_normalizada = modelo_restante.predict(ultima_secuencia)
    prediccion = scaler.inverse_transform(prediccion_normalizada)
    
    # Agregar las predicciones a la lista de resultados
    resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 838ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf  # Asegúrate de importar TensorFlow
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps = 2
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=128, activation='relu'), input_shape=(n_steps, n_features)))  # Aumentado a 128 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, activation='relu', input_shape=(n_steps, n_features)))  # Reducido a 16 unidades
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=100, callbacks=[early_stopping_top], verbose=0)  # Aumentado a 100 épocas

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar y predecir para un producto específico con el modelo restante
def entrenar_y_predecir_rest(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=10, callbacks=[early_stopping_rest], verbose=0)  # Reducido a 10 épocas

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-(n_steps + 1):-1].reshape((1, n_steps, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

for producto in productos_restantes:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_rest(aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12m2.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 546ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Exception ignored in: <function AtomicFunction.__del__ at 0x000001C2D7038900>
Traceback (most recent call last):
  File "c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py", line 300, in __del__
    RUNTIME_FUNCTION_REFS.pop(key)
KeyboardInterrupt: 
Exception ignored in: <function AtomicFunction.__del__ at 0x000001C2D7038900>
Traceback (most recent call last):
  File "c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py", line 300, in __del__
    RUNTIME_FUNCTION_REFS.pop(key)
Keyb

: 

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 18  # Ventana de tiempo de 36 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=128, activation='relu'), input_shape=(n_steps_top, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, activation='relu', input_shape=(n_steps_rest, n_features)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=100, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-(n_steps_top + 1):-1].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=10, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-(n_steps_rest + 1):-1].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstm12-24m.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LeakyReLU
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 24  # Ventana de tiempo de 36 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps_top, n_features)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, input_shape=(n_steps_rest, n_features)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=100, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_top:].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=10, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_rest:].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmdrop.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 760ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━

In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LeakyReLU, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 24  # Ventana de tiempo de 24 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 30 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_30_productos = ventas_totales_por_producto.nlargest(30).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps_top, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, input_shape=(n_steps_rest, n_features)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_top:].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, batch_size=64, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_rest:].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 30
for producto in top_30_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_30_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmdrop1.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 671ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 606ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LeakyReLU, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 24  # Ventana de tiempo de 24 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 45 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_100_productos = ventas_totales_por_producto.nlargest(100).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps_top, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, input_shape=(n_steps_rest, n_features)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_top:].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, batch_size=64, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_rest:].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 45
for producto in top_100_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_45_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/drop5.csv", index=False)


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 638ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LeakyReLU, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 24  # Ventana de tiempo de 24 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 150 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_45_productos = ventas_totales_por_producto.nlargest(45).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps_top, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, input_shape=(n_steps_rest, n_features)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_top:].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, batch_size=64, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_rest:].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 45
for producto in top_45_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_150_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmb.csv", index=False)

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


c:\Users\vgARCIARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, LeakyReLU, Activation
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Lista para acumular las filas de resultados
resultados_por_producto = []

# Elegir un número de pasos de tiempo
n_steps_top = 24  # Ventana de tiempo de 24 meses para productos top
n_steps_rest = 12  # Ventana de tiempo de 12 meses para productos restantes
n_features = 1  # Cambia esto si tienes más características

# Calcular las ventas totales por producto y obtener los 150 productos con mayores ventas
ventas_totales_por_producto = sell_in_completo.groupby("product_id")["tn"].sum()
top_45_productos = ventas_totales_por_producto.nlargest(45).index

# Función para crear secuencias de tiempo
def crear_secuencias(datos, n_steps, step_ahead=1):
    X, y = [], []
    for i in range(len(datos)):
        end_ix = i + n_steps
        out_end_ix = end_ix + step_ahead - 1
        if out_end_ix > len(datos) - 1:
            break
        seq_x, seq_y = datos[i:end_ix], datos[out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Inicializar scaler
scaler = MinMaxScaler()

# Definir la función para construir el modelo para productos top
def build_top_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(units=256, return_sequences=True), input_shape=(n_steps_top, n_features)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=128)))
    model.add(Dropout(0.3))
    model.add(Dense(64))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Definir la función para construir el modelo para productos restantes
def build_rest_model():
    model = Sequential()
    model.add(LSTM(units=16, input_shape=(n_steps_rest, n_features)))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse')
    return model

# Inicializar EarlyStopping para los modelos top
early_stopping_top = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Inicializar EarlyStopping para los modelos restantes
early_stopping_rest = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Función para entrenar y predecir para un producto específico con el modelo top
def entrenar_y_predecir_top(aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_normalizado, n_steps_top, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_top_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=200, batch_size=32, callbacks=[early_stopping_top], verbose=0)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_top:].reshape((1, n_steps_top, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Función para entrenar el modelo único para productos restantes
def entrenar_modelo_rest(productos_restantes):
    try:
        # Agrupar datos de todos los productos restantes
        aux_combined = []
        for producto in productos_restantes:
            aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
            aux_normalizado = scaler.fit_transform(aux)
            aux_combined.append(aux_normalizado)

        aux_combined = np.concatenate(aux_combined, axis=0)

        # Crear secuencias de tiempo
        X, y = crear_secuencias(aux_combined, n_steps_rest, step_ahead=2)
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # Construir el modelo
        model = build_rest_model()

        # Entrenar el modelo con todos los datos
        model.fit(X, y, epochs=50, batch_size=64, callbacks=[early_stopping_rest], verbose=0)

        return model
    except Exception as e:
        print(f"Error al entrenar el modelo para los productos restantes: {str(e)}")
        return None

# Función para predecir para un producto específico con el modelo de productos restantes
def predecir_rest(model, aux, producto):
    try:
        # Normalizar los datos
        aux_normalizado = scaler.fit_transform(aux)

        # Preparar los datos para la predicción
        ultima_secuencia = aux_normalizado[-n_steps_rest:].reshape((1, n_steps_rest, n_features))

        # Predecir las ventas para el próximo período
        prediccion_normalizada = model.predict(ultima_secuencia)

        # Desnormalizar la predicción
        prediccion = scaler.inverse_transform(prediccion_normalizada)

        # Agregar las predicciones a la lista de resultados
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': prediccion[0][0]})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {str(e)}")
        resultados_por_producto.append({'product_id': producto, 'prediccion_1': 0})

# Procesar los productos top 45
for producto in top_45_productos:
    aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
    entrenar_y_predecir_top(aux, producto)

# Procesar los productos restantes
productos_restantes = sell_in_completo[~sell_in_completo["product_id"].isin(top_45_productos)]["product_id"].unique()

# Entrenar el modelo único para los productos restantes
modelo_rest = entrenar_modelo_rest(productos_restantes)

# Realizar predicciones individuales para cada producto restante con el modelo único
if modelo_rest is not None:
    for producto in productos_restantes:
        aux = sell_in_completo[sell_in_completo["product_id"] == producto].drop(columns=["product_id"]).values
        predecir_rest(modelo_rest, aux, producto)

# Convertir la lista de resultados en un DataFrame
predicciones_por_producto = pd.DataFrame(resultados_por_producto)

# Mostrar el DataFrame con las predicciones por producto
print(predicciones_por_producto)

# Guardar el DataFrame con las predicciones por producto en un archivo CSV
predicciones_por_producto.to_csv("C:/Users/vgarciario/desktop/UA MASTER/labo3/lstmdrop4.csv", index=False)
